<a href="https://colab.research.google.com/github/JasminiSantos/TDE3-Apache-Spark-SQL/blob/main/TDE3_Apache_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=5e4def85ce211e23fcd548f4ebcbc5baad5d21139b6020ee68cfe287a5a38561
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!wget https://jpbarddal.github.io/assets/data/bigdata/transactions_amostra.csv.zip
!unzip transactions_amostra.csv.zip

--2023-06-03 13:56:05--  https://jpbarddal.github.io/assets/data/bigdata/transactions_amostra.csv.zip
Resolving jpbarddal.github.io (jpbarddal.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to jpbarddal.github.io (jpbarddal.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47513871 (45M) [application/zip]
Saving to: ‘transactions_amostra.csv.zip’

transactions_amostr 100%[===================>]  45.31M   219MB/s    in 0.2s    

2023-06-03 13:56:07 (219 MB/s) - ‘transactions_amostra.csv.zip’ saved [47513871/47513871]

Archive:  transactions_amostra.csv.zip
  inflating: transactions_amostra.csv  
  inflating: __MACOSX/._transactions_amostra.csv  


In [3]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
.master('local[*]')\
.appName('tde3').getOrCreate()

df = spark.read.csv("transactions_amostra.csv", header=True, inferSchema=True, sep=";")

df.createOrReplaceTempView("transactions")

In [ ]:
# ======= QUESTÃO 1 =======

# CONTAR QUANTAS TRANSAÇÕES TEM O BRASIL ENVOLVIDO
transacoesBrasil = spark.sql("SELECT COUNT(*) AS count FROM transactions WHERE country_or_area = 'Brazil'")

# BUSCA A QUANTIDADE
contador = transacoesBrasil.first()["count"]

print("NÚMERO DE TRANSAÇÕES ENVOLVENDO O BRASIL:", contador)

In [ ]:
# ======= QUESTÃO 2 =======

# CALCULAR O NUMERO DE TRANSAÇÕES POR TIPO DE FLOW E ANO
transacoesPorFlowAno = spark.sql("SELECT Flow, Year, COUNT(*) AS count FROM transactions GROUP BY Flow, Year")

# PRINTA OS RESULTADOS
transacoesPorFlowAno.show()

In [6]:
# ======= QUESTÃO 3 =======

# CALCULANDO AS MÉDIAS DOS COMODITIES REFERENTE A CADA ANO
mediaComoditiesPorAno = spark.sql("SELECT year, AVG(trade_usd) AS average_value FROM transactions GROUP BY year")

# PRINTA
mediaComoditiesPorAno.show()

+----+--------------------+
|year|       average_value|
+----+--------------------+
|1990| 1.172426586778952E7|
|2003|1.3028917611334749E7|
|2007|2.3710673174875777E7|
|2015|   3.1115574884196E7|
|2006|2.1175872541099638E7|
|2013| 3.306115128882995E7|
|1997|   9549881.214776853|
|1988| 1.864297055638571E7|
|1994|1.1350325049077941E7|
|2014| 4.612040441345007E7|
|2004|1.5388487793083541E7|
|1991| 1.306922385515173E7|
|1996|1.1945524161286663E7|
|1989|1.1263871329920229E7|
|1998|1.0175610459598826E7|
|2012|3.9028921881444596E7|
|2009|2.5068409504465386E7|
|2016| 2.941832757526777E7|
|1995|1.2286454103356835E7|
|2001|   9942220.288239626|
+----+--------------------+
only showing top 20 rows



In [7]:
# ======= QUESTÃO 4 =======

# CALCULA O PREÇO MÉDIO POR TIPO DE UNIDADE, ANO E CATEGORIA QUE SÃO DO TIPO EXPORTAÇÃO NO BRASIL
# E AGRUPO POR ANO E CATEGORIA
precoMedioPorUnidadeAnoCat = spark.sql("""
    SELECT Year, category, AVG(trade_usd) AS average_price
    FROM transactions
    WHERE Flow = 'Export' AND country_or_area = 'Brazil'
    GROUP BY  Year, Category
""")

# PRINTA
precoMedioPorUnidadeAnoCat.show()

+----+--------------------+-------------------+
|Year|            category|      average_price|
+----+--------------------+-------------------+
|2007|30_pharmaceutical...|            95057.0|
|1993|27_mineral_fuels_...|          5313712.5|
|2016|37_photographic_o...| 2319.6666666666665|
|2000|     01_live_animals|          1952949.0|
|1992|15_animal_vegetab...|          2108165.4|
|2008|38_miscellaneous_...|       4.49661584E7|
|2014|68_stone_plaster_...|          1.84138E7|
|2006|27_mineral_fuels_...| 3407871.6666666665|
|1993|28_inorganic_chem...|         4578559.24|
|2008|28_inorganic_chem...|         4449268.25|
|1994|39_plastics_and_a...| 3172772.5454545454|
|2005|02_meat_and_edibl...|       2.09560584E8|
|2007|48_paper_paperboa...|5.556047253333333E7|
|1991|93_arms_and_ammun...|            69896.0|
|1999|80_tin_and_articl...|        1.5815404E7|
|1994|17_sugars_and_sug...|8.347317433333333E7|
|1997|60_knitted_or_cro...|         3641387.75|
|1999|22_beverages_spir...|          203

In [21]:
# ======= QUESTÃO 5 =======

# CALCULA O VALOR MÁXIMO, MÍNIMO E MÉDIA POR TIPO DE UNIDADE E ANO
maxMinMedia = spark.sql("""
    SELECT Year, quantity_name, AVG(trade_usd) AS valorMedio, MAX(trade_usd) AS valorMaximo, MIN(trade_usd) AS valorMinimo
    FROM transactions
    GROUP BY Year, quantity_name
    ORDER BY valorMedio, valorMaximo, valorMinimo
""")

maxMinMedia.show()

+----+--------------------+------------------+-----------+-----------+
|Year|       quantity_name|        valorMedio|valorMaximo|valorMinimo|
+----+--------------------+------------------+-----------+-----------+
|2002|  Number of packages|             550.0|        550|        550|
|2006|  Number of packages|            1268.0|       1268|       1268|
|1991|Electrical energy...|            2515.0|       2515|       2515|
|2016|    Length in metres|  74562.9512195122|     961206|         19|
|2015|    Length in metres|116034.65909090909|    1631741|         13|
|2012|  Number of packages| 144575.2857142857|     459884|       6961|
|2003|  Thousands of items|          402348.0|    2428778|        721|
|2004|  Number of packages|          436233.5|     815330|      57137|
|2013|    Length in metres| 678828.8545454545|   26620892|         77|
|2014|    Length in metres| 758637.3518518518|   18030313|         15|
|1989|    Length in metres|1013384.7647058824|   27013051|        518|
|2010|

In [9]:
# ======= QUESTÃO 6 =======

# PROCURA O PAÍS COM MAIOR MÉDIA E PRINTA SOMENTE O MAIOR
paisComMaiorMedia = spark.sql("""
    SELECT country_or_area, AVG(trade_usd) AS avg_price
    FROM transactions
    WHERE Flow = 'Export'
    GROUP BY country_or_area
    ORDER BY avg_price DESC
    LIMIT 1
""")

paisComMaiorMedia.show()

+---------------+--------------------+
|country_or_area|           avg_price|
+---------------+--------------------+
|         Angola|1.636966606814285...|
+---------------+--------------------+



In [19]:
# ======= QUESTÃO 7 =======

# ENCONTRA A MAIOR QUANTIDADE QUE APARECE O COMMODITIE EM CADA TIPO DE FLOW
# E RETORNA O MAIOR EM CADA
comoditieMaisComercializado = spark.sql("""
    SELECT Flow, MAX(total_amount) AS max_amount
    FROM (
        SELECT Flow, Commodity, SUM(quantity) AS total_amount
        FROM transactions
        WHERE Year = '2016' AND Flow IN ('Export', 'Import', 'Re-Export', 'Re-Import')
        GROUP BY Flow, Commodity
    ) temp
    GROUP BY Flow
""")
comoditieMaisComercializado.show()


+---------+----------------+
|     Flow|      max_amount|
+---------+----------------+
|   Import|2.58289373308E11|
|   Export|3.79546246752E11|
|Re-Export|      1.261968E9|
|Re-Import|     3.8774873E7|
+---------+----------------+

